$$
\DeclareMathOperator\tr{tr}
\DeclareMathOperator\Tr{Tr}
\DeclareMathOperator\dev{dev}
\DeclareMathOperator\Dev{Dev}
\DeclareMathOperator\sph{sph}
\DeclareMathOperator\Sph{Sph}
\newcommand{\bF}{\mathbf{F}}
\newcommand{\bC}{\mathbf{C}}
\newcommand{\bd}{\mathbf{d}}
\newcommand{\bc}{\mathbf{c}}
\newcommand{\bA}{\mathbf{A}}
\newcommand{\bS}{\mathbf{S}}
\newcommand{\Bsigma}{{\boldsymbol{\sigma}}}
\newcommand{\pdiff}[2]{\frac{\partial#1}{\partial#2}}
\nonumber
$$

# General Setup

In [ ]:
%matplotlib nbagg

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.comp import IntegrationRuleSpace
from netgen.csg import *
from ngsolve.solvers import Newton

In [ ]:
SetNumThreads(1)

cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
geo = CSGeometry()
geo.Add(cube)

mesh = Mesh(geo.GenerateMesh(maxh=0.2, quad_dominated=False))
#Draw(mesh)

names = [ "bottom", "back", "right", "front", "top", "left", "inner" ]
for i, name in enumerate(names):
    mesh.ngmesh.SetBCName(i, name)
print (mesh.GetBoundaries())

d = mesh.dim

In [ ]:
import numpy as np

# Parameters

In [ ]:
parameters = dict(
    G=1,
    Gprime_f=1e2,
)

In [ ]:
G = Parameter(parameters["G"])
Gprime_f = Parameter(parameters["Gprime_f"])
Gprime = Gprime_f * G

# Energy functions

In [ ]:
I = Id(d)

In [ ]:
def C(F):
    return F.trans * F


def I_1(C):
    return Trace(C)


def I_3(C):
    return Det(C)


# "Deviatoric" part of energy density
def Psi_dev(C, G):
    return G/2 * (I_1(C) - d - log(I_3(C)))


# Purely volumetric part of energy density
def Psi_vol(C, G_prime):
    return G_prime/2 * (sqrt(I_3(C)) - 1)**2


def Psi(C, G, G_prime):
    return Psi_dev(C, G) + Psi_vol(C, G_prime)

In [ ]:
def Compile(cf, **options): 
    _realcompile = options.get("realcompile", False)
    return cf.Compile(realcompile=_realcompile)

# A homogeneous BVP

In [ ]:
fes_u = VectorH1(mesh, order=2, dirichletx="front", dirichlety="left", dirichletz="top|bottom")

In [ ]:
u, u_test = fes_u.TnT()

In [ ]:
def F(u):
    return I + Grad(u)

gfu = GridFunction(fes_u)

# Forms

In [ ]:
energy_density = Psi(C(F(u)), G, Gprime)

In [ ]:
a = BilinearForm(fes_u, symmetric=True)
a += Variation(Compile(energy_density) * dx)

In [ ]:
t = 0
gfu.Set((0, 0, 0))
delta_t = Parameter(0.1)

# Run

In [ ]:
u_top = Parameter(0)
tol = 1e-10
t = 0
gfu.Set((0, 0, 0))
dfu = GridFunction(fes_u)

In [ ]:
def disp(t):
    return t

In [ ]:
while t < 1:
    
    with TaskManager(pajetrace=10**8):
        
        print()
        print("t: ", t)
        print("disp(t): ", disp(t))
        dfu.Set((dfu[0], dfu[1], disp(t)), definedon=mesh.Boundaries("top"))
        ret = Newton(a, gfu, inverse="pardiso", dirichletvalues=dfu.vec)
        print(ret)
        t += delta_t.Get()

In [ ]:
Draw(gfu[2], mesh)